# Amazon Bedrock Knowledge Base and Agent

In [1]:
!pip install python-dotenv packaging

In [2]:
# Import boto3 and os
import boto3
import os

# Load environment
from dotenv import load_dotenv
load_dotenv("env.txt")

True

# Define Amazon Bedrock Agent and AWS Secrets Manager use boto3 client, also get AWS Account ID

In [3]:
# Define Amazon Bedrock Agent and AWS Secrets Manager use boto3 client
bedrockagent = boto3.client('bedrock-agent', region_name="us-west-2")
secretmanager = boto3.client('secretsmanager', region_name="us-west-2")

# Get AWS Account ID
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

# Define IAM Role for Bedrock Knowledge Base, also get Pinecone host and AWS Secret Manager ARN

In [4]:
# Define IAM Role for Bedrock Knowledge Base
bedrockkbiam = f"arn:aws:iam::{account_id}:role/service-role/AmazonBedrockExecutionRoleForKnowledgeBase_a6qi8"
# Get Pinecone host
pineconehost = os.getenv('PINECONE_HOST')

# Get AWS Secret Manager ARN
response = secretmanager.get_secret_value(SecretId='pineconeapikey')
secretarn = response['ARN']

# Define Vector Knowledge Base Configuration

In [5]:
vectorKnowledgeBaseConfiguration = {
    'embeddingModelArn': "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0",  # embedding model using Amazon Titan Embedding
    'embeddingModelConfiguration': {
        'bedrockEmbeddingModelConfiguration': {
            'dimensions': 512,
            'embeddingDataType': 'FLOAT32'
        }
    }
}

# Define Pinecone Configuration

In [6]:
pineconeConfiguration = {
    'connectionString': pineconehost,  # Pinecost connection host
    'credentialsSecretArn': secretarn,  # AWS Secret Manager ARN
    'fieldMapping': {
        'metadataField': 'metadata',
        'textField': 'text'
    }
}

# Create Bedrock Knowledge Base using Vector Knowledge Base and Pinecone Configuration

In [7]:
knowledgebase = bedrockagent.create_knowledge_base(
    name="financial-and-data-expert",
    roleArn=bedrockkbiam,
    knowledgeBaseConfiguration={
        'type': 'VECTOR',
        'vectorKnowledgeBaseConfiguration': vectorKnowledgeBaseConfiguration
    },
    storageConfiguration={
        'type': 'PINECONE',
        'pineconeConfiguration': pineconeConfiguration
    }
)

# Get Bedrock Knowledge Base ID for Next Step

In [8]:
knowledgebaseId = knowledgebase['knowledgeBase']['knowledgeBaseId']
print(knowledgebaseId)

SNOHDWIYR4


# Define Amazon S3 Configuration

In [9]:
s3Configuration = {
    'bucketArn': 'arn:aws:s3:::all-in-bedrock',  # store knowledge base data
    'bucketOwnerAccountId': account_id  # AWS Account ID
}

# Create Data Source using Amazon S3 Configuration

In [10]:
datasource = bedrockagent.create_data_source(
    name="s3-bucket-knowledge-base",
    knowledgeBaseId=knowledgebaseId,
    dataSourceConfiguration={
        'type': 'S3',
        's3Configuration': s3Configuration
    }
)

# Get Data Source ID

In [11]:
datasourceId = datasource['dataSource']['dataSourceId']
print(datasourceId)

RG5FLSDZ5A


# Start Ingestion Job / Sync between Data Source and Knowledge Base

In [12]:
sync = bedrockagent.start_ingestion_job(
    dataSourceId=datasourceId,
    knowledgeBaseId=knowledgebaseId
)
print("Sync was successfully.")

Sync was successfully.


# Define Amazon Bedrock Agent Runtime, also get Amazon Nova model Micro version

In [13]:
bedrockagentruntime = boto3.client("bedrock-agent-runtime", region_name="us-west-2")
model = f"arn:aws:bedrock:us-west-2:{account_id}:inference-profile/us.amazon.nova-micro-v1:0"

# Create Invoke / Test Knowledge Base Function

In [14]:
def invokekb(input, model, knowledgebaseId):
    response = bedrockagentruntime.retrieve_and_generate(
        input={
            'text': input
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': knowledgebaseId,
                'modelArn': model
            }
        },
    )
    return response

# Try ask related to Knowledge Base

In [15]:
input = "How do I transfer funds to another savings account?"
knowledgebaseId = knowledgebaseId

response = invokekb(input, model, knowledgebaseId)
generated_text = response['output']['text']
print(generated_text)

To transfer funds to another savings account, follow these steps:

1. Log into your online banking account.
2. Navigate to the Transfers section.
3. Enter the recipient's account details, including the account number and routing number.
4. Confirm the transfer details and submit the request.

Please note that the transfer typically processes within 1 business day. If you need to transfer funds immediately, consider using a wire transfer, which may have additional fees. Always check with your bank for any specific requirements or fees associated with the transfer.


In [16]:
input = "Why was my loan application denied?"
knowledgebaseId = knowledgebaseId

response = invokekb(input, model, knowledgebaseId)
generated_text = response['output']['text']
print(generated_text)

Your loan application may have been denied due to several reasons, including:

1. **Credit Score**: A low credit score can negatively impact your loan application. Lenders typically have minimum credit score requirements to approve loans.

2. **Debt-to-Income Ratio**: If your debt-to-income ratio is too high, it may indicate that you have too much debt relative to your income, which can make you a higher risk for the lender.

3. **Incomplete Documents**: Missing or incomplete documentation can lead to a denial. Ensure that you have provided all required documents, such as proof of income, employment verification, and identification documents.

To get more specific details about your loan application denial, it is recommended to contact your loan team directly at 18005555678. They will be able to provide you with the exact reasons for the denial and may offer guidance on how to improve your chances for future applications.


In [17]:
input = "What product name did Karen Daniels buy?"
knowledgebaseId = knowledgebaseId

response = invokekb(input, model, knowledgebaseId)
generated_text = response['output']['text']
print(generated_text)

Based on the search results provided, Karen Daniels purchased "Snap-A-Way Black Print Carbonless Ruled Speed Letter, Triplicate". This product falls under the category of "Office Supplies" and "Paper" in the transaction details.


# Try ask unrelated to Knowledge Base

In [18]:
input = "What is AWS?"
knowledgebaseId = knowledgebaseId

response = invokekb(input, model, knowledgebaseId)
generated_text = response['output']['text']
print(generated_text)

AWS, or Amazon Web Services, is a comprehensive and widely adopted cloud platform that offers a wide range of on-demand computing services and features. It provides various services such as computing power, storage, databases, analytics, machine learning, and more, allowing businesses and individuals to build, deploy, and manage applications and services in a scalable and flexible manner. AWS is known for its reliability, security, and global reach, making it a popular choice for organizations of all sizes. With AWS, users can pay only for the resources they consume, making it a cost-effective solution for businesses.


# Create Bedrock Agent

In [22]:
instruction = '''
You are a financial customer service expert who will answer all questions from customers.
You are also a data analyst who can analyze and show answer from user.
You must answer based on the knowledge base without detailed explanation.
If you get a question that is not related to the knowledge base, you answer "I don't know."
You don't use "\\n" for answer, instead use "." for answer.
'''.strip()
agentResourceRoleArn = f"arn:aws:iam::{account_id}:role/service-role/AmazonBedrockExecutionRoleForAgents_TPIC81GO4M"

In [23]:
agent = bedrockagent.create_agent(
    agentName="financial-and-data-agent",
    agentResourceRoleArn=agentResourceRoleArn,
    agentCollaboration='DISABLED',
    foundationModel='us.amazon.nova-micro-v1:0',
    instruction=instruction,
    orchestrationType='DEFAULT',
    description="Financial customer service expert and data analyst agent"
)
print("Your agent was ready")

Your agent was ready


In [24]:
agentId = agent['agent']['agentId']
print(agentId)

SSYVT7QT09


# Associate Knowledge Base to Bedrock Agent

In [27]:
description = '''
If you get a question that is not related to the knowledge base, you answer "I don't know."
You don't use "\\n" for answer, instead use "." for answer.
'''.strip()

In [28]:
associatekb = bedrockagent.associate_agent_knowledge_base(
    agentId=agentId,
    agentVersion="DRAFT",
    description=description,
    knowledgeBaseId=knowledgebaseId,
    knowledgeBaseState='ENABLED'
)
print("Association knowledge base is finished.")

Association knowledge base is finished.


In [29]:
prepareagent = bedrockagent.prepare_agent(
    agentId=agentId
)
print("Prepare agent was successfully prepared.")

Prepare agent was successfully prepared.


# Create Bedrock Agent Alias

In [30]:
createalias = bedrockagent.create_agent_alias(
    agentAliasName="firstAlias",
    agentId=agentId,
    description="First version of alias agent."
)
print("Finished.")

Finished.


In [31]:
agentAliasId = createalias['agentAlias']['agentAliasId']
print(agentAliasId)

TOQPSBELCZ


# Create Invoke / Test Bedrock Agent Function

In [32]:
import uuid
sessionId=str(uuid.uuid4())
print(sessionId)

d9d40fb6-e6f9-4404-88c7-a69760c7db89


In [33]:
def invokeagent(input):
    response = bedrockagentruntime.invoke_agent(
        agentAliasId=agentAliasId,
        agentId=agentId,
        bedrockModelConfigurations={
            'performanceConfig': {
                'latency': 'standard'
            }
        },
        enableTrace=False,
        endSession=False,
        sessionId=sessionId,
        inputText=input
    )
    return response

In [34]:
input = "How do I download my checking statements from the mobile app?"
response = invokeagent(input)
print(response)

{'ResponseMetadata': {'RequestId': '771c98d5-b146-4704-abcb-1e4b530323ef', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 06 Jun 2025 03:18:20 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '771c98d5-b146-4704-abcb-1e4b530323ef', 'x-amz-bedrock-agent-session-id': 'd9d40fb6-e6f9-4404-88c7-a69760c7db89', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 1}, 'contentType': 'application/json', 'sessionId': 'd9d40fb6-e6f9-4404-88c7-a69760c7db89', 'completion': <botocore.eventstream.EventStream object at 0x7fe7aba076d0>}


# Extract 'completion' from Bedrock Agent response

In [37]:
def invoke_agent(input):
    response = bedrockagentruntime.invoke_agent(
        agentId=agentId,
        agentAliasId=agentAliasId,
        sessionId=sessionId,
        inputText=input,
    )

    completion = ""

    for event in response.get("completion"):
        completion = completion + event["chunk"]["bytes"].decode()

    return completion

In [38]:
agentanswer = invoke_agent("How do I download my checking statements from the mobile app?")
print(agentanswer)

To download your checking statements from the mobile app, follow these steps:
Open the app, go to Accounts, select your account, tap Statements, choose the month, and click Download PDF. Ensure you have a PDF viewer.


In [39]:
agentanswer = invoke_agent("Why was my loan application denied?")
print(agentanswer)

Denials for loan applications may occur due to credit score, debt-to-income ratio, or incomplete documents. To get more details, you can contact our loan team at 18005555678.


In [40]:
agentanswer = invoke_agent("What are the fees for your investment service?")
print(agentanswer)

Our investment service charges a 0.5 percent annual fee based on assets. No additional trading fees for standard portfolios. For a detailed schedule, you can contact us at 18005554321.


In [41]:
agentanswer = invoke_agent("Can I change the payment due date for my credit card?")
print(agentanswer)

Yes, you can change the payment due date for your credit card. To request a change, you can contact our credit card team at 18005556789. Some restrictions may apply based on terms.


In [42]:
agentanswer = invoke_agent("What product name did Karen Daniels buy?")
print(agentanswer)

Karen Daniels bought "Snap-A-Way Black Print Carbonless Ruled Speed Letter, Triplicate".


In [43]:
agentanswer = invoke_agent("What product name did Budiono Santoso buy?")
print(agentanswer)

Budiono Santoso bought "New Paper High Quality".


In [48]:
agentanswer = invoke_agent("What sub-category of Bush Somerset Collection Bookcase?")
print(agentanswer)

The sub-category of the Bush Somerset Collection Bookcase is Bookcases.


In [49]:
agentanswer = invoke_agent("What is AWS?")
print(agentanswer)

Sorry, I cannot answer about AWS as it is not within my knowledge base.


In [50]:
agentanswer = invoke_agent("How to related between AI and cloud computing?")
print(agentanswer)

Sorry, I cannot answer about the relationship between AI and cloud computing as it is not within my knowledge base.
